# Croisement des données 

##### *Claire Toffano-Nioche, I2BC, Paris-Saclay*

##### *Pauline Francois, Anses, Lyon*

##### *Selon les ressources de Matthias Zytnicki, inrae, Toulouse*

##### *et toute la team Roscoff*

## Objectifs du cours

- Permettre de croiser des données génomiques de types différents
- Renforcer les notions vues précedemment
- Se familiariser avec <code>bedtools</code>

Ce que ce cours n'est pas :
- Une réponse à une vraie question biologique
- Une méthode statistiquement valide

## Croisement de données

#### Qu’est-ce c'est ?
- Une comparaison des positions ou intervalles génomiques: 
    - un variant par rapport à des gènes d’intérêt, 
    - des pics de ChIP-Seq par rapport à des gènes différentiellement exprimés, etc.
    
#### À quel moment est-ce valide ?
- Lorsque vous cherchez des co-occurrences
- Lorsque vous donnez des distributions (de distance)

#### À quel moment est-ce douteux ?
- Lorsque les résultats sont présentés comme significatifs.

#### Données disponibles
- Une liste de gènes différentiellement exprimés (Cellules avec un KO du gène beta-2-microglobulin, infectées par l'herpes simplex virus type 1 vs non infectées) (TXT)
- Une liste des sites de fixation de H3K4me3 déterminés par Chip-seq (BED)
- Une liste de variants (VCF)
- Un fichier d'annotation humain (GFF3)
- La liste des chromosomes humains et leur taille (LEN)

## Avant de commencer

1. Vérifier que vous êtes dans votre espace projet :

In [55]:
pwd

/shared/ifbstor1/projects/primir_orfevre_2/tp_croisement


2. Créer un dossier nommé <code>tp_croisement</code> et vous déplacer dedans

In [1]:
mkdir -p tp_croisement
cd tp_croisement
pwd


/shared/ifbstor1/projects/form_2022_32/TP_croisement/tp_croisement


## Accès aux données du TP

1. Regarder la liste des fichiers présents dans <code>/shared/projects/form_2021_26/data/atelier_croisement/</code>

In [42]:
ls /shared/projects/form_2022_32/data/atelier_croisement/

chrs.len		     differentially_expressed_genes.txt
common_all_20180418_div.vcf  h3k4me3_k562.bed


Ce TP repose sur 4 jeux de données, issues d'analayses précédentes ainsi que sur les annotations du génome humain : 

- la longueur des chrs (et des contigs) de la version GRCh27 du génome humain (<code>*.len</code>)
- une liste de SNP variants (<code>*.vcf</code>)
- une liste des identifiants de gènes différentiellement exprimés (<code>*.txt</code>)
- une liste des pics issus d'une analyse de ChIP-seq (<code>*.bed</code>), ici une marque d'histone
- les annotations du génome humain (<code>*.gff3</code>)

2. Ces données sont "loin" (/shared/...). Pour simplifier l'écriture des lignes de commandes, on va utiliser la commande <code>ln -s</code> qui permet de créer des raccourcis. 
Exécuter la liste de commandes suivantes et remplir les commentaires (lignes commençant par "#") :

In [44]:
#lien gènes différentiellement exprimés
ln -s /shared/projects/form_2022_32/data/atelier_croisement/differentially_expressed_genes.txt DEgenes.txt
#lien fichier variants
ln -s /shared/projects/form_2022_32/data/atelier_croisement/common_all_20180418_div.vcf variant.vcf
#lien bed de pics Chip-seq
ln -s /shared/projects/form_2022_32/data/atelier_croisement/h3k4me3_k562.bed picChip.bed
#lien contigs 
ln -s /shared/projects/form_2022_32/data/atelier_croisement/chrs.len .
#lien gff3
ln -s /shared/bank/homo_sapiens/GRCh38/gff3/Homo_sapiens.GRCh38.94.gff3 hsGRCh38.gff3

ls -lh

total 20K
lrwxrwxrwx 1 cpatiou cpatiou 62 Nov 17 09:07 chrs.len -> /shared/projects/form_2022_32/data/atelier_croisement/chrs.len
lrwxrwxrwx 1 cpatiou cpatiou 88 Nov 17 09:07 DEgenes.txt -> /shared/projects/form_2022_32/data/atelier_croisement/differentially_expressed_genes.txt
lrwxrwxrwx 1 cpatiou cpatiou 65 Nov 17 09:07 hsGRCh38.gff3 -> /shared/bank/homo_sapiens/GRCh38/gff3/Homo_sapiens.GRCh38.94.gff3
lrwxrwxrwx 1 cpatiou cpatiou 70 Nov 17 09:07 picChip.bed -> /shared/projects/form_2022_32/data/atelier_croisement/h3k4me3_k562.bed
lrwxrwxrwx 1 cpatiou cpatiou 81 Nov 17 09:07 variant.vcf -> /shared/projects/form_2022_32/data/atelier_croisement/common_all_20180418_div.vcf


3. Afin de se familiariser avec le contenu des données, on affiche les 2 premières lignes de l'ensemble des fichiers :

In [56]:
head -n 2 *

==> chrs.len <==
1	248956422
10	133797422

==> DEgenes.txt <==
ENSG00000004846
ENSG00000005981

==> hsGRCh38.gff3 <==
##gff-version 3
##sequence-region   1 1 248956422

==> picChip.bed <==
22	16192349	16192565	region_1
22	16846630	16870710	region_2

==> variant.vcf <==
##fileformat=VCFv4.0
##fileDate=20180418


4. Pour deux des fichiers on ne voit que des commentaires (souvent des lignes qui commencent par "#") ; afficher les deux dernières lignes de ces fichiers :

In [57]:
tail -n 2 hsGRCh38.gff3 variant.vcf

==> hsGRCh38.gff3 <==
Y	havana	exon	56855244	56855488	.	+	.	Parent=transcript:ENST00000431853;Name=ENSE00001794473;constitutive=1;ensembl_end_phase=-1;ensembl_phase=-1;exon_id=ENSE00001794473;rank=1;version=1
###

==> variant.vcf <==
Y	26614668	rs376925794	TC	T	.	.	RS=376925794;RSPOS=26614671;dbSNPBuildID=138;SSR=0;SAO=0;VP=0x050000000005000002000200;WGT=1;VC=DIV;ASP;CAF=0.9968,0.003244;COMMON=1
Y	26624486	rs771929773	T	TA	.	.	RS=771929773;RSPOS=26624486;dbSNPBuildID=144;SSR=0;SAO=0;VP=0x050000000005040024000200;WGT=1;VC=DIV;ASP;VLD;KGPhase3;CAF=0.9968,0.003244;COMMON=1


5. Afficher l'arborescence des fichiers avec la commande tree.
Observer les liens symboliques :

In [58]:
tree

.
├── chrs.len -> /shared/projects/form_2022_32/data/atelier_croisement/chrs.len
├── DEgenes.txt -> /shared/projects/form_2022_32/data/atelier_croisement/differentially_expressed_genes.txt
├── hsGRCh38.gff3 -> /shared/bank/homo_sapiens/GRCh38/gff3/Homo_sapiens.GRCh38.94.gff3
├── picChip.bed -> /shared/projects/form_2022_32/data/atelier_croisement/h3k4me3_k562.bed
└── variant.vcf -> /shared/projects/form_2022_32/data/atelier_croisement/common_all_20180418_div.vcf

0 directories, 5 files


6. Activer les outils bedtools (v2.30.0) et bc (1.07.1)

In [59]:
module load bedtools/2.30.0 bc/1.07.1

## Problème

##### *Question scientifique*

Mes gènes différentiellement exprimés contiennent-ils plus de pics de chip qu'attendu ? (oui/non, pas de quantification)

En d'autres termes, y a-t-il plus ou moins de pics de chip-seq dans les gènes différentiellement exprimés par rapport aux gènes non différentiellement exprimés ?

##### *Données*
- DEgenes.txt
- picChip.bed
- hsGRCh38.gff3

In [60]:
head -n 5 DEgenes.txt picChip.bed
tail -n 5 hsGRCh38.gff3

==> DEgenes.txt <==
ENSG00000004846
ENSG00000005981
ENSG00000006747
ENSG00000015568
ENSG00000047457

==> picChip.bed <==
22	16192349	16192565	region_1
22	16846630	16870710	region_2
22	17067019	17067283	region_3
22	17070620	17114004	region_4
22	17128021	17136091	region_5
Y	.	biological_region	26627457	26628186	0.997	+	.	external_name=rank %3D 1;logic_name=firstef
Y	havana	pseudogene	56855244	56855488	.	+	.	ID=gene:ENSG00000235857;Name=CTBP2P1;biotype=processed_pseudogene;description=C-terminal binding protein 2 pseudogene 1 [Source:HGNC Symbol%3BAcc:HGNC:23940];gene_id=ENSG00000235857;logic_name=havana;version=1
Y	havana	pseudogenic_transcript	56855244	56855488	.	+	.	ID=transcript:ENST00000431853;Parent=gene:ENSG00000235857;Name=CTBP2P1-201;biotype=processed_pseudogene;tag=basic;transcript_id=ENST00000431853;transcript_support_level=NA;version=1
Y	havana	exon	56855244	56855488	.	+	.	Parent=transcript:ENST00000431853;Name=ENSE00001794473;constitutive=1;ensembl_end_phase=-1;ensembl_phase=

<code>DEgenes.txt</code> est notre liste de gènes DE :

![](images/liste_geneDE.png)

.

.

<code>picChip.bed</code> est notre liste de pics issus du chip-seq et positionnés sur le génome :

![](images/liste_chip.png)

.

.

Pour croiser ces données il faut positionner les gènes DE sur le génome, on fait cela grâce au fichier d'annotation (gènes non DE plus clairs) :

![](images/genes_position_genome.png)

.

.

Le positionnement des données sur le génome permet maintenant de les croiser :

![](images/associationGeneDE_chip.png)

.

.

Le but sera de compter le nombre gène dans chacune des 4 classes possibles :

![](images/geneDE_Chip.png)

##### *Protocole envisagé*
<details><summary></summary>

  A. Extraire les intervalles génomiques des gènes différentiellement exprimés (DE)
    
  B. Comparer les intervalles génomiques des gènes DE avec les régions H3K4me3 (ie. l'emplacement des pics de chip-seq)
  
  C. Compter le nombre de chevauchements entre les gènes DE et les pics

  D. Trouver les gènes non-différentiellement exprimés

  E. Comparer ces intervalles génomiques avec les régions H3K4me3

  F. Compter le nombre de chevauchements entre les gènes non DE et les pics

  G. Comparer les nombres de chevauchements

</details>

.

.

.

#### A. Extraire les intervalles génomiques des gènes différentiellement exprimés (DE)

1. Chercher une liste de mots dans un texte et rediriger la sortie standard dans un fichier nommé <code>DEgenes_all.gff3</code>

In [71]:
grep -f DEgenes.txt hsGRCh38.gff3 > DEgenes_all.gff3
wc -l DEgenes.txt
wc -l DEgenes_all.gff3
wc -l hsGRCh38.gff3
head DEgenes_all.gff3

162 DEgenes.txt
1106 DEgenes_all.gff3
2814993 hsGRCh38.gff3
1	ensembl_havana	gene	33081104	33120530	.	+	.	ID=gene:ENSG00000142920;Name=AZIN2;biotype=protein_coding;description=antizyme inhibitor 2 [Source:HGNC Symbol%3BAcc:HGNC:29957];gene_id=ENSG00000142920;logic_name=ensembl_havana_gene;version=16
1	havana	lnc_RNA	33081104	33093400	.	+	.	ID=transcript:ENST00000478635;Parent=gene:ENSG00000142920;Name=AZIN2-211;biotype=processed_transcript;transcript_id=ENST00000478635;transcript_support_level=3;version=5
1	ensembl_havana	mRNA	33081113	33120394	.	+	.	ID=transcript:ENST00000294517;Parent=gene:ENSG00000142920;Name=AZIN2-201;biotype=protein_coding;ccdsid=CCDS375.1;tag=basic;transcript_id=ENST00000294517;transcript_support_level=1;version=10
1	havana	lnc_RNA	33081148	33120529	.	+	.	ID=transcript:ENST00000481886;Parent=gene:ENSG00000142920;Name=AZIN2-212;biotype=processed_transcript;transcript_id=ENST00000481886;transcript_support_level=2;version=5
1	havana	lnc_RNA	33081150	33120526	.	+	.	I

2. Se restreindre aux gènes et rediriger la sortie standard dans un fichier nommé <code>DEgenes.gff3</code>

In [75]:
awk -F"\t" 'BEGIN {OFS=FS} $3 == "gene" {print}' "DEgenes_all.gff3" > DEgenes.gff3 
cut -f 3 DEgenes.gff3 | sort | uniq
head DEgenes.gff3

gene
1	ensembl_havana	gene	33081104	33120530	.	+	.	ID=gene:ENSG00000142920;Name=AZIN2;biotype=protein_coding;description=antizyme inhibitor 2 [Source:HGNC Symbol%3BAcc:HGNC:29957];gene_id=ENSG00000142920;logic_name=ensembl_havana_gene;version=16
1	ensembl_havana	gene	70852353	71047808	.	-	.	ID=gene:ENSG00000050628;Name=PTGER3;biotype=protein_coding;description=prostaglandin E receptor 3 [Source:HGNC Symbol%3BAcc:HGNC:9595];gene_id=ENSG00000050628;logic_name=ensembl_havana_gene;version=20
1	ensembl_havana	gene	74198235	74544393	.	+	.	ID=gene:ENSG00000259030;Name=FPGT-TNNI3K;biotype=protein_coding;description=FPGT-TNNI3K readthrough [Source:HGNC Symbol%3BAcc:HGNC:42952];gene_id=ENSG00000259030;logic_name=ensembl_havana_gene;version=7
1	ensembl_havana	gene	111722064	111755824	.	-	.	ID=gene:ENSG00000284755;Name=AL049557.1;biotype=protein_coding;description=inka box actin regulator 2 [Source:NCBI gene%3BAcc:55924];gene_id=ENSG00000284755;logic_name=ensembl_havana_gene;version=1
1	ensembl_ha

Ou, en plus court, générer directement le fichier <code>DEgenes.gff3</code> (à l'aide d'un pipe) :

In [74]:
grep -f DEgenes.txt hsGRCh38.gff3 > DEgenes_all.gff3 | awk -F"\t" 'BEGIN {OFS=FS} $3 == "gene" {print}' "DEgenes_all.gff3" > DEgenes.gff3

.


#### B. Comparer les intervalles génomiques des gènes DE avec les régions H3K4me3 (ie. l'emplacement des pics de chip-seq)

Comme on va utiliser la suite d'outils que propose [Bedtools](https://bedtools.readthedocs.io/en/latest/content/bedtools-suite.html), avant tout, faisons un tour sur la documentation.

Chaque sous-commande fonctionne sur le même principe :

<code>bedtools [sub-command] [options]</code>

Les options varient selon la commande. Il suffit de cliquer sur son nom sur le site pour avoir accès à la fonction de la commande, à ses options et à des exemples.

Les trois qui nous intéresserons dans un premier temps sont [<code>intersect</code>](https://bedtools.readthedocs.io/en/latest/content/tools/intersect.html), [<code>flank</code>](https://bedtools.readthedocs.io/en/latest/content/tools/flank.html) et [<code>closest</code>](https://bedtools.readthedocs.io/en/latest/content/tools/closest.html)

Bedtools accepte quasiment tous les formats de fichiers tabulés (GFF3, BED, etc)

Dans notre cas de figure, nous souhaitons savoir si des gènes DE contiennent des pics et obtenir le nom de ces gènes. Nous utilisons donc la sous-commande "bedtools ....." ?

Le fichier A sera celui de l'annotation des gènes DE (<code>DEgenes.gff3</code>) et le B correspondra aux pics (<code>picChip.bed</code>). Quelle option choisir pour obtenir uniquement les annotations du fichier A qui croisent avec le fichier B ?

Rediriger la sortie standard dans un fichier nommé <code>intersect_DEgenes_picChip.gff3</code>

In [88]:
bedtools intersect -a DEgenes.gff3 -b picChip.bed -wa  -u > intersect_DEgenes_picChip.gff3

.

#### C. Compter le nombre de chevauchements entre les gènes DE et les pics

Nous avons obtenu la liste des gènes dans lesquels se trouvent des pics (<code>intersect_DEgenes_picChip.gff3</code>). Nous avons maintenant besoin de les compter.

In [89]:
wc -l intersect_DEgenes_picChip.gff3

77 intersect_DEgenes_picChip.gff3


.


#### D. Trouver des gènes non-différentiellement exprimés

1. Sélectionner tous les gènes présents dans l'annotation et les écrire dans un fichier nommé <code>all_genes.gff3</code>

In [79]:
awk -F"\t" 'BEGIN {OFS=FS} $3 == "gene" {print}' "hsGRCh38.gff3" > all_genes.gff3 
cut -f 3 all_genes.gff3 | sort | uniq
wc -l all_genes.gff3

gene
21492 all_genes.gff3


2. Ne garder que les gènes qui ne sont pas dans le fichier des gènes DE (regarder l'option -v) dans un fichier <code>notDEgenes.gff3</code>

In [91]:
bedtools intersect -a all_genes.gff3 -b DEgenes.gff3 -v > notDEgenes.gff3
wc -l notDEgenes.gff3

21180 notDEgenes.gff3


.

.

.

.

.
L'option la plus importante pour cette question est donc l'option <code>-v</code>. Mais par défaut, <code>intersect</code> valide une interscetion dès qu'il y a une base en commun entre les 2 régions. Dans ce cas d'usage où l'on traite des gènes en entier, il suffit que quelques gènes se chevauchent pour qu'ils soient exclus à tort (c'est le cas pour 4 gènes dans ces données). La façon correcte est donc de préciser que l'on veut valider les chevauchements que si les gènes se chevauchent à 100% entre eux. Les options pour spécifier un % de chavauchement (exprimés entre <code>[0-1]</code>, 1 pour 100%) sont <code>-F</code> pour le fichier A et <code>-f</code> pour le fichier B. Il faut aussi que les 2 gènes soient sur le même brin (option <code>-s</code>). Du coup, une commande plus exacte est :

In [96]:
bedtools intersect -a all_genes.gff3 -b DEgenes.gff3 -v -f 1 -F 1 -s -wa > notDEgenes.gff3
wc -l notDEgenes.gff3

21330 notDEgenes.gff3


#### E.  Comparer ces intervalles génomiques avec les régions H3K4me3
Faire l'intersection entre le fichier gff3 des gènes non DE et les pics de Chip-seq et écrire la sortie dans un fichier <code>intersect_notDEgenes_picChip.gff3</code>

In [97]:
bedtools intersect -a notDEgenes.gff3 -b picChip.bed -wa -u > intersect_notDEgenes_picChip.gff3
wc -l intersect_notDEgenes_picChip.gff3

9438 intersect_notDEgenes_picChip.gff3


.


#### F. Compter le nombre de chevauchements entre les gènes non DE et les pics

Afin de pouvoir ensuite faire des ratios "nb de gène avec au moins un chevauchements/nb de gène" nous devons connaitre les valeurs suivantes :
- Nombre de gènes DE
- Nombre de gènes DE avec chevauchement
- Nombre de gènes non DE
- Nombre de gènes non DE avec chevauchement


In [98]:
wc -l DEgenes.gff3
wc -l intersect_DEgenes_picChip.gff3
wc -l notDEgenes.gff3
wc -l intersect_notDEgenes_picChip.gff3

162 DEgenes.gff3
77 intersect_DEgenes_picChip.gff3
21330 notDEgenes.gff3
9438 intersect_notDEgenes_picChip.gff3


.

#### G. Comparer les nombres de chevauchements

 
 Ratios | avec pics | sans pic | total
-----------|----------|----------|-------
gene DE    |77        |162-77    |162
gene non DE|9438      |21330-9438|21330    
total      |          |          |21492

Sous Bash (commande <code>bc</code>), un calcul peut être réalisé de la façon suivante :
<code>echo "CALCUL" | bc -l</code>

C'est à dire que nous affichons le calcul sur la sortie standard que nous envoyons (pipe) dans la commande <code>bc</code> avec l'option <code>-l</code>. Cette option permet de faire le calcul à l'aide de la bibliothèque mathématique standard.

In [101]:
echo "162-77" | bc -l
echo "21330 - 9438" | bc -l

85
11892


.

.

.

## Bonus n°1

##### *Question scientifique*

Quels sont les variants qui sont dans les promoteurs (2kb en amont du gène) des gènes différentiellement exprimés ?

##### *Données*
- DEgenes.gff3 (généré précédemment)
- variant.vcf
- chrs.len

In [102]:
tail DEgenes.gff3 variant.vcf

==> DEgenes.gff3 <==
9	ensembl_havana	gene	41131306	41199261	.	-	.	ID=gene:ENSG00000215126;Name=CBWD6;biotype=protein_coding;description=COBW domain containing 6 [Source:HGNC Symbol%3BAcc:HGNC:31978];gene_id=ENSG00000215126;logic_name=ensembl_havana_gene;version=10
9	ensembl_havana	gene	64369394	64413142	.	+	.	ID=gene:ENSG00000172014;Name=ANKRD20A4;biotype=protein_coding;description=ankyrin repeat domain 20 family member A4 [Source:HGNC Symbol%3BAcc:HGNC:31982];gene_id=ENSG00000172014;logic_name=ensembl_havana_gene;version=12
9	ensembl_havana	gene	83623049	83644130	.	+	.	ID=gene:ENSG00000148057;Name=IDNK;biotype=protein_coding;description=IDNK%2C gluconokinase [Source:HGNC Symbol%3BAcc:HGNC:31367];gene_id=ENSG00000148057;logic_name=ensembl_havana_gene;version=15
9	ensembl_havana	gene	100442271	100577636	.	+	.	ID=gene:ENSG00000251349;Name=MSANTD3-TMEFF1;biotype=protein_coding;description=MSANTD3-TMEFF1 readthrough [Source:HGNC Symbol%3BAcc:HGNC:38838];gene_id=ENSG00000251349;logic_name=

##### *Protocole envisagé*
<details><summary></summary>

  A. Extraire la région située à 2kb en amont des gènes différentiellement exprimés (DE)
    
  B. Trouver tous les variants chevauchant les régions trouvées précedemment

</details>

.



Nous allons avoir besoin d'une autre sous-commande de bedtools : [<code>bedtools flank</code>](https://bedtools.readthedocs.io/en/latest/content/tools/flank.html) Regardez bien ses options.

Notons qu'il y a une option obligatoire, <code>-g</code> pour communiquer à l'outil la taille des chromosomes. Pourquoi ?

Lancer la commande et écrire la sortie standard dans un fichier <code>flank2kDEgenes.gff3</code>

In [108]:
bedtools flank -i DEgenes.gff3 -g chrs.len -l 2000 -r 0 > flank2kDEgenes.gff3
head flank2kDEgenes.gff3

1	ensembl_havana	gene	33079104	33081103	.	+	.	ID=gene:ENSG00000142920;Name=AZIN2;biotype=protein_coding;description=antizyme inhibitor 2 [Source:HGNC Symbol%3BAcc:HGNC:29957];gene_id=ENSG00000142920;logic_name=ensembl_havana_gene;version=16
1	ensembl_havana	gene	70850353	70852352	.	-	.	ID=gene:ENSG00000050628;Name=PTGER3;biotype=protein_coding;description=prostaglandin E receptor 3 [Source:HGNC Symbol%3BAcc:HGNC:9595];gene_id=ENSG00000050628;logic_name=ensembl_havana_gene;version=20
1	ensembl_havana	gene	74196235	74198234	.	+	.	ID=gene:ENSG00000259030;Name=FPGT-TNNI3K;biotype=protein_coding;description=FPGT-TNNI3K readthrough [Source:HGNC Symbol%3BAcc:HGNC:42952];gene_id=ENSG00000259030;logic_name=ensembl_havana_gene;version=7
1	ensembl_havana	gene	111720064	111722063	.	-	.	ID=gene:ENSG00000284755;Name=AL049557.1;biotype=protein_coding;description=inka box actin regulator 2 [Source:NCBI gene%3BAcc:55924];gene_id=ENSG00000284755;logic_name=ensembl_havana_gene;version=1
1	ensembl_havana	

Maintenant, c'est du déjà-vu : Trouver tous les variants chevauchant les régions trouvées précedemment. Renvoyer la sortie standard dans un fichier nommé <code>intersect_flank2kDEgenes_variant.txt</code>

N'oubliez pas de regarder les options qui nous intéressent [ici](https://bedtools.readthedocs.io/en/latest/content/tools/intersect.html) et visualisez les 5 premières lignes du fichier quand celui ci est généré

In [110]:
bedtools intersect -a variant.vcf -b flank2kDEgenes.gff3 -wa -u > intersect_flank2kDEgenes_variant.txt
head -n 5 intersect_flank2kDEgenes_variant.txt
wc -l intersect_flank2kDEgenes_variant.txt

1	33079702	rs10712676	GA	G	.	.	RS=10712676;RSPOS=33079703;dbSNPBuildID=137;SSR=0;SAO=0;VP=0x0501000a0005170126000200;GENEINFO=AZIN2:113451|LOC105378635:105378635;WGT=1;VC=DIV;SLO;INT;R5;ASP;VLD;G5A;G5;GNO;KGPhase3;CAF=0.8706,0.1294;COMMON=1;TOPMED=0.96872610856269113,0.03127389143730886
1	70850638	rs545762042	AT	A	.	.	RS=545762042;RSPOS=70850639;dbSNPBuildID=142;SSR=0;SAO=0;VP=0x050000080005170026000200;GENEINFO=LOC102724572:102724572;WGT=1;VC=DIV;INT;ASP;VLD;G5A;G5;KGPhase3;CAF=0.7636,0.2364;COMMON=1;TOPMED=0.69237385321100917,0.30762614678899082
1	70850755	rs55931738	TC	T	.	.	RS=55931738;RSPOS=70850756;dbSNPBuildID=137;SSR=0;SAO=0;VP=0x05000008000515003e000200;GENEINFO=LOC102724572:102724572;WGT=1;VC=DIV;INT;ASP;VLD;G5;KGPhase1;KGPhase3;CAF=0.752,0.248;COMMON=1;TOPMED=0.80174885321100917,0.19825114678899082
1	70850755	rs796218873	TC	T	.	.	RS=796218873;RSPOS=70850758;dbSNPBuildID=146;SSR=0;SAO=0;VP=0x050000080005000002000200;GENEINFO=LOC102724572:102724572;WGT=1;VC=DIV;INT;ASP;CAF=0.7

Pour aller plus loin avec bedtools <code>intersect</code>, on peut se demander combien y a-t-il de SNP par promoteur ? 

Stocker l'information dans un fichier <code>intersect_flanck2kDEgenes_n_variant.txt</code> et visualiser les 5 premières lignes

.

.

.

## Bonus n°2

##### *Question scientifique*

Quels sont les gènes différentiellement exprimés les plus proches des pics ChIP ET qui contiennent une mutation ?

##### *Données*
- DEgenes.gff3 (généré dans le problème 1)
- variant.vcf
- picChip.bed

In [ ]:
tail DEgenes.gff3 variant.vcf picChip.bed

##### *Protocole envisagé*
<details><summary></summary>

  1. Trouver les pics qui contiennent une mutation
    
  2. Trouver le gène le plus proche de chaque région précédemment trouvée

</details>

.

.

.

#### Trouver les pics qui contiennent une mutation

Rien de neuf sous le soleil, nous stockons l'output dans un fichier <code>picVariant.bed</code>.

.

#### Trouver le gène le plus proche de chaque région précédemment trouvée

Nous avons besoin d'une nouvelle sous-commande : [<code>bedtools closest</code>](https://bedtools.readthedocs.io/en/latest/content/tools/closest.html)

Attention, bien prendre note du message suivant :
![](images/exo3_graph1.png)

L'output est à stocker dans un fichier nommé <code>picVariant_sorted.bed</code>

Nous tentons un essai en paramètre par défaut. Rediriger la sortie dans un fichier <code>DEgenes_closest_picVariant.bed</code>

Et si on prenait en compte le brin ?

Afficher le nombre de ligne